# Final Project (Team3)

This project aims to figure out unique patterns of crime incidents in Washington DC by visualizing spatial representations on a map and analyzing geographical attributes.

## Part Ⅰ. [WEB] Download Raw data 

- All datasets were gathered from [Open Data DC](https://opendata.dc.gov/explore?collection=Dataset) and preprocessed. 

- There are THREE types of specific datasets that were used in this project
1. CRIME INCIDENTS (5 years: ‘19~23): *.Shp, *CSV
2. Properties(13: Schools, Gas station, etc.): *.Shp
3. Census Economic Characteristics(78 Statistics): *.Shp

## Part Ⅱ. [ArcPro] Make Initial GDB- 'MyProject_pre.gdb'

  - Batch Import (or add feature) *.shp files
  - Show Featrues in the Map, Edit Point Symbology 
  - Arcpy: SET GDB Environment, Make KDE, etc <br>

### 1. Verify GDB 

**Below code is to verify GDB which was generated on ArcPro**
`arcpy.env.workspace` was set to the path for `MyProject_pre.gdb`.

In [2]:
import arcpy

In [3]:
path = r'.\MyProject_pre.gdb'  
arcpy.env.workspace = (path)
arcpy.overwriteOutput = True
#\\apporto.com\dfs\GWU\Users\nanminwoo_gwu\Desktop\IGSUP_data_student\IGSUP_data_student\@Final Project\MyProject (GDB,ArcPy)

In [5]:
# Check Feature classes
fc_list = arcpy.ListFeatureClasses()
print(fc_list)

['Crime_Incidents_in_2023_kml_point', 'Crime_Incidents_in_2023_shp', 'Census_Economic_shp', 'Census_Economic_kml_Polygons', 'Bank_Locations', 'Farmers_Market_Locations', 'Gas_Stations', 'Grocery_Store_Locations', 'Hospitals', 'Hotels', 'Liquor_Licenses', 'Metro_Station_Entrances_in_DC', 'National_Parks', 'Roads', 'Shopping_Centers', 'Sidewalk_Cafe', 'Crime_Incidents_in_2022', 'Crime_Incidents_in_2021', 'Crime_Incidents_in_2020', 'Crime_Incidents_in_2019', 'DC_Public_Schools', 'Crime_Incidents_19_23_Merged', 'Crime_Incidents_19_23_Merged_CSV']


### 2. Census_Economic Feature class
  
   - Basic Check: descrie, field, row counts, samples

In [ ]:
desc2 = arcpy.Describe("Census_Economic_shp")
desc2 

In [ ]:
for fld in desc2.fields:
    #    print("\t" + fld.name)
    print(f"{fld.name} : {fld.Type}")

# Number of record: GetCount : 206 Census Track Areas
count = arcpy.GetCount_management('Census_Economic_shp')
print(f"Name: Census_Econmic_variales, feature count: {count}")

In [ ]:
### 2. Census_Economic (2020 version)
desc2 = arcpy.Describe("Census_Economic_shp")
desc2 
for fld in desc2.fields:
    #    print("\t" + fld.name)
    print(f"{fld.name} : {fld.Type}")
# Number of record: GetCount : 206 Census Track Areas
count = arcpy.GetCount_management('Census_Economic_shp')
print(f"Name: Census_Econmic_variales, feature count: {count}")
# Check the sample: DP03_0009PE (truncated:DP03_0009P): unemployment_rate, DP03_0119PE(DP03_0119P): pct_all_families_below_poverty
from arcpy.da import *
with arcpy.da.SearchCursor("Census_Economic_shp",("DP03_0009P", "DP03_0119P")) as cursor:
    i= 0
    for val in cursor: 
            while i < 5:
                print(f'Unemployment(%): {val[0]}')
                print(f'Poverty(%): {val[1]}')
                i += 1   



## Part Ⅲ. Generate New Features

 - To enrich attributes of crime incidents, a feature engineering was performed as follows. 
 - In this stage, CRIME INCIDENTS.CSV (Same data but *.CSV format) was additionally used. (Because Timestamp variables were truncked in *.Shp)
 - Six features were generated as below.
 - **The subsequent analysis will rely on an aggregated data frame, specifically a feature class within the GDB, encompassing five years' worth of crime incidents.**  
  
> *1. Crime_Type: Recategorized as 4 types* 
>   *(Theft, Other_Property_Crimes, Robbery, Other_Violent_Crimes)* 
> 
> *2. Clear_Time: Time to clear a crime incident(Minutes)*
>   *(END_DATE - START_DATE)*
> 
> *3. Pandemic Phase: 3 Phases*
>   *(Before: 2019.1~2020.2, Lockdown: 2020.3.~ 2021.7, Post: 2021.8~2323.12)* 
> 
> *4. Season: Spring,Summer,Fall,Winter*
> 
> *5. Date Phase: 1: 'Beginning', 2: 'Middle', 3: 'End'*
> 
> *6. Time hour: 0~23*

In [ ]:
# Read CRIME INCIDENTS (*.CSV) dataset from 2019~2023
path = r'.\Data_update\Crime Raw data(openDC)_CSV'  
import pandas as pd
import os
for filename in os.listdir(path):
    if filename.endswith('.csv'):
        filepath = os.path.join(path, filename)
        df_name = filename.split('.')[0]
        year = df_name[-4:]
        globals()[df_name]= pd.read_csv(filepath)
        globals()[df_name]['Year'] = year # Add year column to distinguish
        print(f'Shape of {df_name}: {globals()[df_name].shape}')

Shape of Crime_Incidents_in_2019: (33950, 26)
Shape of Crime_Incidents_in_2020: (27917, 26)
Shape of Crime_Incidents_in_2021: (28348, 26)
Shape of Crime_Incidents_in_2022: (27211, 26)
Shape of Crime_Incidents_in_2023: (26022, 26)


In [ ]:
# Merge 5 years
# List of dataframes
dfs = [Crime_Incidents_in_2019, Crime_Incidents_in_2020, Crime_Incidents_in_2021, Crime_Incidents_in_2022, Crime_Incidents_in_2023]
Crime_Incidents_19_23 = pd.concat(dfs, ignore_index=True)
Crime_Incidents_19_23.shape

(143448, 26)

In [ ]:
# Verify PK of dataset
pk = ['CCN']  #PK is OBJECTID but it change whenever download it 
temp = Crime_Incidents_19_23.drop_duplicates(subset=pk)
print(f'# of records Before Clean: {Crime_Incidents_19_23.shape[0]}\n After Clean: {temp.shape[0]}')

143448 143384


In [ ]:
import copy
df = copy.deepcopy(temp) # will use CCN dup dropped 

# 1. Crime_Type:  4 types (Theft, Other_Property_Crimes, Robbery, Other_Violent_Crimes)  
df['Crime_Type'] = df['OFFENSE']
df['Crime_Type'] = df['Crime_Type'].replace(['THEFT F/AUTO', 'THEFT/OTHER'], 'Theft')
df['Crime_Type'] = df['Crime_Type'].replace(['Arson', 'Burglary', 'Motor Vehicle Theft'], 'Other_Property_Crimes')
df['Crime_Type'] = df['Crime_Type'].replace(['Robbery'], 'Robbery')
df['Crime_Type'] = df['Crime_Type'].replace(['Assault with a Dangerous Weapon', 'Homicide', 'Sex Abuse'], 'Other_Violent_Crimes')

# 2. Clear_Time: END_DATE - START_DATE(Minutes), Time to clear a crime incident
df['START_DATE'] = pd.to_datetime(df['START_DATE'])
df['END_DATE'] = pd.to_datetime(df['END_DATE'])
df['Clear_Time'] = round((df['END_DATE'] - df['START_DATE']).dt.total_seconds() / 60,1)

# 3. Pandemic Phase: (Before: 2019.1~2020.2, Lockdown: 2020.3.~ 2021.7, Post: 2021.8~2323.12) 
df['REPORT_DAT'] = pd.to_datetime(df['REPORT_DAT'])
df['Pandemic_Phase'] = pd.cut(df['REPORT_DAT'],
                               bins=[pd.Timestamp('2019-01-01'), pd.Timestamp('2020-03-01')
                                     , pd.Timestamp('2021-08-01'), pd.Timestamp('2024-01-01')],
                               labels=['Before', 'Lockdown', 'Post'])

# 4. Season: Spring,Summer,Fall,Winter
df['Season'] = df['REPORT_DAT'].dt.month % 12 // 3 + 1
df['Season'] = df['Season'].map({1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'})

# 5. Date Phase: 1: 'Beginning', 2: 'Middle', 3: 'End'
df['Date_Phase'] = df['REPORT_DAT'].dt.day // 10 + 1
df['Date_Phase'] = df['Date_Phase'].map({1: 'Beginning', 2: 'Middle', 3: 'End'})

# 6. Time hour: 0~23
df['Time_Hour'] = df['START_DATE'].dt.hour
df.dtypes

###  Export dataframe to CSV 

   * Exported CSV file will be added on the current GDB  

In [ ]:
#print(Crime_Incidents_19_23.shape, df.shape)
#
feat = ['CCN','Year', 'Crime_Type', 'Clear_Time', 'Pandemic_Phase', 'Season', 'Date_Phase', 'Time_Hour'] 
# PK+ add features: 
df.to_csv(os.path.join(os.getcwd(),'Crime_Incidents_19_23_add_feat_all.csv'))
#df[feat].to_csv(os.path.join(os.getcwd(),'Crime_Incidents_19_23_add_feat.csv'))

## Part Ⅳ. [ArcPro] Make Final GDB - 'MyProject.gdb'

  - Updated new features by adding generated output(Part Ⅲ CSV) to GDB
  - Also Merged previous 5 year Crime incidents features(based on Shapefiles) into 1 feature
  - Verified the final GDB
  - *Feature class "Crime_Incidents_19_23_Merged_CSV" will be mainly used in upcoming analysis*

*Merging and Import part was commented out because final GDB was already generated when submission*

In [ ]:
# 1) Aggregate current features based on Shapefiles 
# List of Crime shapefiles in current GDB
feature_classes = ['Crime_Incidents_in_2019', 'Crime_Incidents_in_2020', 'Crime_Incidents_in_2021', 'Crime_Incidents_in_2022', 'Crime_Incidents_in_2023_shp']  #2023: test(kml,shp,etc)

# [Finished] Define Output merged feature class 
# out_feature_class = "Crime_Incidents_19_23_Merged"
# Used the Merge tool to merge 5 years features (But in memory, Should do Copy to save)
# arcpy.Merge_management(feature_classes, out_feature_class)

In [ ]:
# Check a result, Field names
desc = arcpy.Describe("Crime_Incidents_19_23_Merged")
for fld in desc.fields:
    #    print("\t" + fld.name)
    print(f"{fld.name} : {fld.Type}")
print(df.columns.to_list())  

In [ ]:
# 2) Make "Crime_Incidents_19_23_Merged_CSV" by importing generated output(Part Ⅲ CSV)

# [Finished] Import the CSV file as a table
# arcpy.TableToTable_conversion("Crime_Incidents_19_23_add_feat_all.csv", arcpy.env.workspace, "crime_19_23_csv")

# [Worked on ArcPro] Converted the table to the Point feature class on ArcPro
# 1. On the Map tab, in the Layer group, click the Add Data drop-down menu, and click XY Point Data.
# 2. Choose a table that contains x,y coordinate data.
# 3. Click the X Field drop-down arrow and click the field containing x-coordinate values.
# 4. Click the Y Field drop-down arrow and click the field containing y-coordinate values.


# Alternative trials by arcpy-- 
# arcpy.MakeTableView_management(in_csv, "csv_view_2")
# arcpy.management.XYTableToPoint("csv_view_2", "Crime_Incidents_19_23_Merged_CSV", "X", "Y")
# arcpy.AddJoin_management("Crime_Incidents_19_23_Merged", "CCN", "csv_view_add_feat_f", "CCN")

## Part Ⅴ. Analysis 
   - (Interactive Work between ArcPy and ArcGIS)

In [2]:
import arcpy
path = r'.\MyProject.gdb'    # previous gdb (before update crime variables)
arcpy.env.workspace = (path)
arcpy.overwriteOutput = True

### 1. Crime Incidents Pattern (KDE Hotspot)

#### Step 1. Basic Check: descrie, field, row counts, samples

In [3]:
desc = arcpy.Describe("Crime_Incidents_19_23_Merged_CSV")
#desc = arcpy.Describe("Crime_Incidents_in_2023_kml_point")
desc

catalogPath,"\\apporto.com\dfs\GWU\Users\nanminwoo_gwu\Desktop\IGSUP_data_student\IGSUP_data_student\@Final Project\MyProject (GDB,ArcPy)\MyProject.gdb\Crime_Incidents_19_23_Merged_CSV"
dataType,FeatureLayer


In [4]:
for fld in desc.fields:
    #    print("\t" + fld.name)
    print(f"{fld.name} : {fld.Type}")

# field_list = arcpy.ListFields("Crime_Incidents_in_2023_shp")
# field_list

OBJECTID_1 : OID
Shape : Geometry
Field1 : Integer
X : Double
Y : Double
CCN : Integer
REPORT_DAT : String
SHIFT : String
METHOD : String
OFFENSE : String
BLOCK : String
XBLOCK : Double
YBLOCK : Double
WARD : Double
ANC : String
DISTRICT : Double
PSA : Double
NEIGHBORHOOD_CLUSTER : String
BLOCK_GROUP : String
CENSUS_TRACT : Double
VOTING_PRECINCT : String
LATITUDE : Double
LONGITUDE : Double
BID : String
START_DATE : String
END_DATE : String
OBJECTID : Integer
OCTO_RECORD_ID : String
Year : Integer
Crime_Type : String
Clear_Time : Double
Pandemic_Phase : String
Season : String
Date_Phase : String
Time_Hour : Double


In [5]:
# Numer of record: GetCount
count = arcpy.GetCount_management('Crime_Incidents_19_23_Merged_CSV')
print(f"Name: Crime incidents(5years), feature count: {count}")

Name: Crime incidents(5years), feature count: 143384


Time stamp variales are truncated. So, they will be manually added after converting into Clearance_time

In [6]:
# Check the sample
from arcpy.da import *
with arcpy.da.SearchCursor("Crime_Incidents_19_23_Merged_CSV", ("START_DATE", "END_DATE")) as cursor:
    i = 0
    for val in cursor:
        if i < 5:
            print(f'START: {val[0]}')
            print(f'END: {val[1]}')
            i += 1
        else:
            # Add your desired action here when i reaches 5 or more
            print("Reached 5 iterations. Do something else here.")
            break 

START: 2019-05-16 23:15:24+00:00
END: 2019-05-16 23:18:13+00:00
START: 2019-05-18 10:31:59+00:00
END: 2019-05-18 11:04:48+00:00
START: 2019-05-18 18:30:47+00:00
END: 2019-05-18 18:35:33+00:00
START: 2019-04-02 23:20:46+00:00
END: None
START: 2019-04-04 05:00:17+00:00
END: 2019-04-04 05:50:06+00:00
Reached 5 iterations. Do something else here.


#### Step 2. Create KDE feature 

- Interactive KDE Plot on ArcGIS by input value as parameter To EXPLORE pattern of Crime  
- Two Input Parameters: Pandemic Phase, Crime Type


In [16]:
# 1. Inactive 'Crime_Incidents_19_23' points
# 2. Delete 'Crime_Incidents_19_23' [temp layer]

Choose Crime Type (1:Theft, 2:Other_Property_Crimes), 3:Robbery, 4:Other_Violent_Crimes1
Theft


In [ ]:
from arcpy.sa import *
arcpy.CheckOutExtension("Spatial")

In [33]:

# 1. ask user to chosse crime type  #Pandemic_Phase # Crime_Type
list = {1:'Theft', 2:'Other_Property_Crimes', 3:'Robbery', 4:'Other_Violent_Crimes'}
crime_type = input("Choose Crime Type (1:Theft, 2:Other_Property_Crimes), 3:Robbery, 4:Other_Violent_Crimes")
index = int(crime_type)
crime_type = list[index]
print(crime_type)

out_layers = "Crime_Incidents_19_23_" + crime_type
arcpy.MakeFeatureLayer_management(in_features="Crime_Incidents_19_23_Merged_CSV"
                                  , out_layer=out_layers)


#2.
new_field = arcpy.AddFieldDelimiters(arcpy.env.workspace, "Crime_Type")
SQLExp = f"{new_field} = '{crime_type}'"  # Use quotes for string comparison

try:
    arcpy.SelectLayerByAttribute_management(in_layer_or_view=out_layers,
                                            selection_type="NEW_SELECTION",
                                            where_clause=SQLExp)
    
    # 3. Make KDE feature by using temporary features

    with arcpy.EnvManager(extent=arcpy.Extent(-77.112318, 38.814667, -76.910140, 38.993573)):
        outKernelDensity2 = KernelDensity(in_features=out_layers,
                                          population_field="",
                                          cell_size=0.005,
                                          search_radius=0.01) 

    # 1. Inactive 'Crime_Incidents_19_23' points
    # 2. Delete 'Crime_Incidents_19_23' [temp layer]
    
    # 4.Refresh selection
#    arcpy.SelectLayerByAttribute_management(in_layer_or_view='Crime_Incidents_19_23',
#                                            selection_type="CLEAR_SELECTION")

except arcpy.ExecuteError:
    print(arcpy.GetMessages())


Choose Crime Type (1:Theft, 2:Other_Property_Crimes), 3:Robbery, 4:Other_Violent_Crimes4
Other_Violent_Crimes


### (TBD) 2. Explore Proximity feature
 
  - Number of Property near crime spots
  - Whether property exists Within a buffer area of each crime incident spot 

In [ ]:
# # Create a buffer around the crime incidents
# arcpy.Buffer_analysis("Crime_incident", "Crime_Buffer", "100 Feet")

# # Intersect the buffer with the properties
# arcpy.Intersect_analysis(["Crime_Buffer", "Properties"], "Buffer_Properties")

# # Calculate the number of properties within each buffer
# arcpy.Statistics_analysis("Buffer_Properties", "Buffer_Eat_drinking", [["OBJECTID", "COUNT"]])


In [ ]:
#### TBD Works (DEC 1 update: GDB, Code, ipynb,py.)
# ex.  [Hospital] num_hospitals_crime: Integer or dummy(whether it exists or not, 0, 1)							
# 1. Calculate: 'num_hospitals' within 100 feet from a crime point 							
# Hospital- Crime Point - SelectLayerByLocation WITHIN_A_DISTANCE (or Buffer_analysis)							
# GetCount(or Statistics_analysis)							
							
# 2. Iteration for all crime points: get spatial features							
							
# 3.  Link with socio-economic data  (summarized zonal statistics, crime intensity)							
# Summarize 'Average of num_hospitals' by Census_tract					** Sum of		
# (End image)							
# Census	Poverty	num_crime	Avg_num_hospital				
# Region A	high	10	1				


# # Create a buffer around the crime incidents
# arcpy.Buffer_analysis("Crime_incident", "Crime_Buffer", "100 Feet")

# # Intersect the buffer with the properties
# arcpy.Intersect_analysis(["Crime_Buffer", "Properties"], "Buffer_Properties")

# # Calculate the number of properties within each buffer
# arcpy.Statistics_analysis("Buffer_Properties", "Buffer_Eat_drinking", [["OBJECTID", "COUNT"]])

In [ ]:
# #### 4. Create Symbols- Numeric represent of Census variale on the Map (TBD)
# # Join the economic characteristics to the census tracts
# arcpy.JoinField_management("Census_Tracts", "TRACT_ID", "Economic_Characteristics", "TRACT_ID", "DP03_0009PE")

# # Classify the unemployment rate
# arcpy.ApplySymbologyFromLayer_management("Census_Tracts", "Unemployment_Symbology.lyrx")

# Reference for opensource work
# Read *gdb
# import geopandas as gpd
# # Read the .gdb file
# gdf = gpd.read_file('project.gdb')
